# Generation of QA Pairs Notebook

## Notebook Overview

This notebook leverages NLP to generate, evaluate, and refine question–answer (QA) pairs from cleantech media and patent documents, with the goal of advancing retrieval-augmented generation (RAG) systems for cleantech.

To achieve this, we employed a series of  NLP techniques and evaluation strategies:

1. Data Selection & Preparation:
We curated 50–100 relevant paragraphs from cleantech media and patent datasets, ensuring diversity and relevance to key topics..

2. QA Pair Generation:
Using large language models (LLMs), we generated fact-based QA pairs from the selected paragraphs, ensuring close alignment with the source content.

3. QA Pair Classification:
QA pairs were classified into categories (e.g., Factual, Comparative) using rule-based and embedding methods based on semantic similarity to predefined prototypes.

4. LLM-as-a-Judge Evaluation:
Recognizing that not every generated QA pair would be of equal quality, we then evaluated each pair using LLM-based critique agents. These agents assessed three key criteria:
  - Groundedness: How clearly can the question be answered based on the given context?
  - Relevance: How useful is the question for developing cleantech NLP applications?
  - Standalone Clarity: Does the question make sense independently, without needing extra context?
      
  For each criterion, the LLM provided both a numerical score (1 to 5) and detailed feedback, which helped us identify and filter out lower-quality QA pairs.

5. Filtering & Analysis:
QA pairs scoring at least 4 out of 5 on all criteria were retained, and the dataset's quality was visualized for further refinement.

## 1. Introduction & Learning Goals

We began our project with the objective of creating a comprehensive QA dataset from relevant cleantech texts. Our aim was to extract 50–100 high-quality paragraphs from both the Cleantech Media and Google Patent datasets. These paragraphs represent a spectrum of topics, ensuring that the generated questions and answers address various aspects of cleantech innovation.

This notebook guides you through each step of the process of generating high quality QA pairs, reinforcing the importance of data quality and robust evaluation in NLP applications.

## 2. Data Preparation & Visualization

### Setting Your OpenRouter.ai Key

Before starting, we need to set our API key to authenticate with OpenRouter.ai for using our chosen LLM. In a Colab environment, you can store the API key in a file (e.g., .env) so that it is securely loaded into the notebook.

In [11]:
%%writefile .env

api_key = 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



Overwriting .env


We use several Python libraries for data processing, visualization, and NLP. In Colab, you can install these dependencies using !pip install commands. For example:

In [12]:
!pip install -q tiktoken pandas numpy seaborn matplotlib spacy langchain tqdm openai sentence-transformers python-dotenv


### Loading the Dependencies

In [13]:
import os
from dotenv import load_dotenv
import pandas as pd
import numpy as np
import ast
from collections import Counter
import seaborn as sns
import matplotlib.pyplot as plt
import tiktoken
import spacy
from openai import OpenAI
from tqdm.auto import tqdm
from typing import Optional, List, Tuple
import json
import textwrap

from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.docstore.document import Document as LangchainDocument

pd.set_option("display.max_colwidth", None)

nlp = spacy.load("en_core_web_sm")

encoding = tiktoken.encoding_for_model("gpt-4")

import nltk
import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer, PorterStemmer

nltk.download('punkt_tab')
nltk.download('stopwords')
nltk.download('wordnet')

stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()
stemmer = PorterStemmer()




[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


### Setting up our LLM

In [14]:
load_dotenv()

client = OpenAI(
  base_url="https://openrouter.ai/api/v1",
  api_key=os.getenv("api_key"),
)

### Downloading the Dataset

For this project, we work with two main datasets:

- Cleantech Media Dataset: Contains media articles focused on cleantech topics.

- Google Patent Dataset: Provides detailed patent information related to cleantech.

If you’re using Colab, you can download these datasets directly from Kaggle (using the Kaggle API) or from provided Google Drive links. Make sure to store them in a designated folder (e.g., data/bronze).


In [15]:
import kagglehub
from kagglehub import KaggleDatasetAdapter

file_path_media_dataset_v3_2024_10_28 = "cleantech_media_dataset_v3_2024-10-28.csv"

articles_df  = kagglehub.load_dataset(
  KaggleDatasetAdapter.PANDAS,
  "jannalipenkova/cleantech-media-dataset",
  file_path_media_dataset_v3_2024_10_28,
)

file_path_media_dataset_v3_2024_09_20 = "cleantech_rag_evaluation_data_2024-09-20.csv"

human_eval = kagglehub.load_dataset(
    KaggleDatasetAdapter.PANDAS,
    "jannalipenkova/cleantech-media-dataset",
    file_path_media_dataset_v3_2024_09_20,
    pandas_kwargs={
        "sep": ";",
        "engine": "python",
        "encoding": "latin-1",
    }
)

# print("First 5 records:", df_media_dataset_v3_2024_09_20.head())


<ipython-input-15-8dd5952fdf74>:6: DeprecationWarning: load_dataset is deprecated and will be removed in future version.
  articles_df  = kagglehub.load_dataset(
<ipython-input-15-8dd5952fdf74>:14: DeprecationWarning: load_dataset is deprecated and will be removed in future version.
  human_eval = kagglehub.load_dataset(


In [16]:

file_paths = [
    'bq-results-20240124-055833-1706076079048.json',
    'CleanTech_22-24.json',
    'CleanTech_22-24_updated.json'
]

dfs = {}

for file_path in file_paths:
   df = kagglehub.load_dataset(
       KaggleDatasetAdapter.PANDAS,
        "prakharbhandari20/cleantech-google-patent-dataset",
        "bq-results-20240124-055833-1706076079048.json",
        pandas_kwargs={
            "lines": True,
        })
   dfs[file_path] = df



patent_df = dfs['bq-results-20240124-055833-1706076079048.json']

<ipython-input-16-c1df106e5251>:10: DeprecationWarning: load_dataset is deprecated and will be removed in future version.
  df = kagglehub.load_dataset(
<ipython-input-16-c1df106e5251>:10: DeprecationWarning: load_dataset is deprecated and will be removed in future version.
  df = kagglehub.load_dataset(
<ipython-input-16-c1df106e5251>:10: DeprecationWarning: load_dataset is deprecated and will be removed in future version.
  df = kagglehub.load_dataset(


In [17]:
# !curl -O https://raw.githubusercontent.com/tamannaKumavat/SwissText_Shared_Task_2025/main/data/merged_media_qa_pairs.csv
# !curl -O https://raw.githubusercontent.com/tamannaKumavat/SwissText_Shared_Task_2025/main/data/patent_dataset_qa_pairs.csv


### Text Preprocessing


In [18]:
def preprocess_text(text, apply_stemming=False):
    """
    Preprocess a given text by:
      - Converting to lowercase
      - Tokenizing
      - Removing punctuation and stop words
      - Applying lemmatization (and optionally stemming)
    """
    if not isinstance(text, str):
        return []

    text = text.lower()
    tokens = word_tokenize(text)
    tokens = [token for token in tokens if token not in string.punctuation]
    tokens = [token for token in tokens if token not in stop_words]
    tokens = [lemmatizer.lemmatize(token) for token in tokens]

    if apply_stemming:
        tokens = [stemmer.stem(token) for token in tokens]

    return tokens


In [19]:
articles_df.head()

Unnamed: 0  \
0       93320   
1       93321   
2       98159   
3       98158   
4       31128   

                                                                                           title  \
0                                                      XPeng Delivered ~100,000 Vehicles In 2021   
1                                                  Green Hydrogen: Drop In Bucket Or Big Splash?   
2            World’ s largest floating PV plant goes online in China – pv magazine International   
3  Iran wants to deploy 10 GW of renewables over the next four years – pv magazine International   
4                        Eastern Interconnection Power Grid Said ‘ Being Challenged in New Ways’   

         date  author  \
0  2022-01-02     NaN   
1  2022-01-02     NaN   
2  2022-01-03     NaN   
3  2022-01-03     NaN   
4  2022-01-03     NaN   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            

In [20]:
# patent_df = pd.read_csv("/content/drive/MyDrive/SwissText SharedTask/google_patents.csv")
# patent_df = pd.read_csv("google_patents.csv")
patent_df.head()

,publication_number,application_number,country_code,title_localized,abstract_localized,publication_date,inventor,cpc
0,US-2022239235-A1,US-202217717397-A,US,"[{'text': 'Adaptable DC-AC Inverter Drive System and Operation', 'language': 'en', 'truncated': False}]","[{'text': 'Disclosed is an adaptable DC-AC inverter system and its operation. The system includes multiple DC input sources as input to provide a stable operation under various conditions. DC input sources may be added to the system or removed from the system without impacting the functionality of the system. The disclosed system is suited for solar energy harvesting in grid-connected or off-grid modes of operation.', 'language': 'en', 'truncated': False}]",20220728,[],"[{'code': 'H02M7/5395', 'inventive': True, 'first': False, 'tree': []}, {'code': 'H02J3/32', 'inventive': False, 'first': False, 'tree': []}, {'code': 'H02M1/32', 'inventive': True, 'first': False, 'tree': []}, {'code': 'H02J1/10', 'inventive': True, 'first': False, 'tree': []}, {'code': 'H02J3/381', 'inventive': True, 'first': False, 'tree': []}, {'code': 'H02M7/493', 'inventive': True, 'first': True, 'tree': []}, {'code': 'H02J2300/24', 'inventive': False, 'first': False, 'tree': []}, {'code': 'H02J2300/30', 'inventive': False, 'first': False, 'tree': []}]"
1,US-2022239251-A1,US-202217580956-A,US,"[{'text': 'System for providing the energy from a single contiguous solar energy structure to at least two different meters', 'language': 'en', 'truncated': False}]","[{'text': 'In accordance with an example embodiment, a solar energy system comprises: a solar energy structure comprising photovoltaic solar panels contiguously covering an area; a first inverter configured to receive power from a first string of photovoltaic solar panels, wherein the first inverter is configured to provide the power received at the first inverter to a first meter; and a second inverter configured to receive power from a second string of photovoltaic solar panels, wherein the second inverter is configured to provide the power received at the second inverter to a second meter.', 'language': 'en', 'truncated': False}]",20220728,[],"[{'code': 'H02S40/38', 'inventive': True, 'first': False, 'tree': []}, {'code': 'G01R22/061', 'inventive': True, 'first': False, 'tree': []}, {'code': 'B60L53/51', 'inventive': True, 'first': False, 'tree': []}, {'code': 'H02S40/32', 'inventive': True, 'first': True, 'tree': []}]"
2,EP-4033090-A1,EP-21152924-A,EP,"[{'text': 'Verfahren zum steuern einer windenergieanlage', 'language': 'de', 'truncated': False}, {'text': 'Method for controlling a wind energy system', 'language': 'en', 'truncated': False}, {'text': 'Procédé de commande d&#39;une éolienne', 'language': 'fr', 'truncated': False}]","[{'text': 'Verfahren zum Steuern einer Windenergieanlage (100), wobei die Windenergieanlage (100) einen aerodynamischen Rotor (106) aufweist, der mit variabler Drehzahl betrieben wird, und der in ihrem Blattwinkel verstellbare Rotorblätter (108) ausweist, wobei die Windenergieanlage (100) in einem Teillastbereich durch eine Betriebskennliniensteuerung gesteuert wird, die eine Betriebskennlinie verwendet, wobei die Betriebskennlinie (402) einen Zusammenhang zwischen der Drehzahl und einer einzustellenden Generatorzustandsgröße vorgibt, wobei die einzustellende Generatorzustandsgröße eine Generatorleistung oder ein Generatormoment ist, und die Betriebskennliniensteuerung so erfolgt, dass in Abhängigkeit von einer erfassten Drehzahl ein durch die Betriebskennlinie vorgegebener Wert der Generatorzustandsgröße eingestellt wird, und die Windenergieanlage in einem Volllastbereich, durch eine Pitchregelung (304) geregelt wird, bei der die Drehzahl durch Verstellen der Blattwinkel auf einen Drehzahlsollwert geregelt wird, wobei in wenigstens einem vorgebbaren Drehzahlbereich des Teillastbereichs, und/oder in einem Übergangsbereich vom Teillastbereich zum Volllastbereich die Windenergieanlage durch eine Drehzahlleistungsr

In [8]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


#### Tokenize the Title and the Context of the datasets

In [9]:
articles_df['tokens_title'] = articles_df['title'].apply(preprocess_text)
articles_df['tokens_content'] = articles_df['content'].apply(preprocess_text)

In [10]:
patent_df['tokens_title'] = patent_df['title_localized'].apply(
    lambda lst: preprocess_text(lst[0]['text']) if isinstance(lst, list) and lst and 'text' in lst[0] else []
)

patent_df['tokens_abstract'] = patent_df['abstract_localized'].apply(
    lambda lst: preprocess_text(lst[0]['text']) if isinstance(lst, list) and lst and 'text' in lst[0] else []
)

### Functions Definitions

In this section, we perform detailed exploratory data analysis to understand the characteristics of our dataset. Our objectives are to:

  - Overview:

      - Examine the dataset for missing values and duplicate entries.

      - Inspect key columns (e.g., the author column, which may have very few non-null entries).

      - Ensure that our dataset has the right structure before proceeding to more advanced steps.

  - Domain Analysis:

      - Identify which publishers contribute the most content.

      - Analyze publication trends such as counts of articles per domain.

      - Visualize the distribution of articles across different publishers.

  - Cleaning:

      - Normalize the URLs to ensure consistency.

      - Remove duplicate articles that might skew our analysis.

      - Standardize article formats by, for example, removing standardized introductions present in many scraped articles.


By performing these steps, we ensure that our dataset is clean, consistent, and ready for downstream tasks like QA pair generation, embedding creation, and evaluation.

We have implemented several custom plotting functions to help visualize the dataset:

  - show_questions:
  This function prints the first n questions from a list along with a title. It’s useful for getting a quick overview of a sample of generated questions.

  - gpt4_tokenizer:
  Uses the GPT-4 tokenizer from tiktoken to split the text into tokens. This allows us to analyze the token distribution in our questions.

  - plot_character_distribution:
  Plots a histogram (with KDE) of the character lengths for the questions. It includes vertical lines for both the median and mean, letting us understand the typical question length.

  - plot_word_distribution:
  Similar to the character distribution, this function plots the distribution of word counts in the questions and marks the median and mean word counts.

  - plot_gpt4_token_distribution:
  Tokenizes each question using the GPT-4 tokenizer, filters for alphabetic tokens, counts the top 20 most common tokens, and displays them in a horizontal barplot. This helps to identify common words or fragments in your QA pairs.

  - plot_non_stopword_token_distribution:
  Similar to the previous function, but filters out common stopwords using spaCy before counting tokens. This gives a clearer picture of the subject-specific vocabulary in your questions.

  - printwrap:
  A helper function to wrap long text lines for more readable output when printe

In [21]:
# plot functions

def show_questions(title: str, questions: list, n: int = 6) -> None:
    """
    Display the first n questions from the given list with a title.

    Parameters:
    - title: A string to display as the category title.
    - questions: A list of questions.
    - n: The maximum number of questions to display (default is 6).
    """
    print(title)
    print("-" * len(title))
    for idx, question in enumerate(questions[:n], start=1):
        print(f"{idx}. {question}")
    print()



def gpt4_tokenizer(text: str):
    """
    Tokenizes the text using the GPT-4 tokenizer from tiktoken.
    Returns a list of token strings.
    """
    token_ids = encoding.encode(text)
    tokens = [encoding.decode([tid]) for tid in token_ids]
    return tokens

def plot_character_distribution(df):
    """
    Plots a histogram (with KDE) of the character lengths in the 'question' column.
    Displays vertical lines for the median and mean character counts.
    """
    char_lengths = df["question"].map(len)
    sns.histplot(char_lengths, kde=True)
    plt.title("Amount of characters in questions")
    plt.xlabel("Amount of characters")
    plt.ylabel("Number of questions")

    median_char_len = char_lengths.median()
    mean_char_len = char_lengths.mean()
    plt.axvline(median_char_len, color='r', linestyle='--', label=f"Median: {median_char_len:.2f}")
    plt.axvline(mean_char_len, color='g', linestyle='--', label=f"Mean: {mean_char_len:.2f}")
    plt.legend()
    plt.show()

def plot_word_distribution(df):
    """
    Plots a histogram (with KDE) of the word counts in the 'question' column.
    Displays vertical lines for the median and mean word counts.
    """
    word_counts = df["question"].map(lambda x: len(x.split()))
    sns.histplot(word_counts, kde=True)
    plt.title("Amount of words in questions")
    plt.xlabel("Amount of words")
    plt.ylabel("Number of questions")

    median_word_count = word_counts.median()
    mean_word_count = word_counts.mean()
    plt.axvline(median_word_count, color='r', linestyle='--', label=f"Median: {median_word_count:.2f}")
    plt.axvline(mean_word_count, color='g', linestyle='--', label=f"Mean: {mean_word_count:.2f}")
    plt.legend()
    plt.show()

def plot_gpt4_token_distribution(df):
    """
    Uses the GPT-4 tokenizer to tokenize the 'question' column,
    filters out non-alphabetic tokens, counts the top 20 most common tokens,
    and displays a horizontal barplot.
    """
    questions = df['question'].tolist()

    all_tokens = [token for question in questions for token in gpt4_tokenizer(question)]

    alpha_tokens = [token.strip().lower() for token in all_tokens if token.strip().isalpha()]

    alpha_token_counts = Counter(alpha_tokens)
    top_tokens = alpha_token_counts.most_common(20)

    if top_tokens:
        tokens, counts = zip(*top_tokens)
        sns.barplot(x=list(counts), y=list(tokens))
        plt.title("Most common alphabetic tokens")
        plt.xlabel("Count")
        plt.ylabel("Token")
        plt.show()
    else:
        print("No tokens found after filtering.")

def plot_non_stopword_token_distribution(df):
    """
    Uses the GPT-4 tokenizer to tokenize the 'question' column,
    filters out non-alphabetic tokens and then filters out stopwords using spaCy.
    It then counts the top 20 most common non-stopword tokens and displays a horizontal barplot.
    """
    questions = df['question'].tolist()

    all_tokens = [token for question in questions for token in gpt4_tokenizer(question)]
    alpha_tokens = [token.strip().lower() for token in all_tokens if token.strip().isalpha()]
    non_stop_tokens = [token for token in alpha_tokens if not nlp.vocab[token].is_stop]
    non_stop_token_counts = Counter(non_stop_tokens)
    top_tokens = non_stop_token_counts.most_common(20)

    if top_tokens:
        tokens, counts = zip(*top_tokens)
        sns.barplot(x=list(counts), y=list(tokens))
        plt.title("Most common non-stopword tokens")
        plt.xlabel("Count")
        plt.ylabel("Token")
        plt.show()
    else:
        print("No non-stop tokens found after filtering.")



def printwrap(text, wrap_lenght = 80):
  wrapped_text = textwrap.fill(text,wrap_lenght)
  return wrapped_text

**Overview:**
We first examine the dataset for missing values and duplicates to identify any problematic columns like a sparsely populated author field.

**Domain Analysis:**
We analyze the distribution of articles across different publishers. This helps us understand which domains dominate the dataset and informs decisions such as stratified subsampling.

**Cleaning**:
Normalizing URLs and removing duplicate entries ensures consistency and reduces noise in the data. We standardize article formats by removing redundant or standardized sections often introduced by web scraping.

**Plotting Functions:**
Our custom plot functions help visualize:

How long the questions are in terms of characters and words.

The most common tokens both overall and non-stopword tokens using the GPT-4 tokenizer.

### Selection of important Paragraphs
For generating robust QA pairs, we first needed to identify the most important and informative paragraphs within our dataset. To do this, we explored the entire dataset and manually selected a set of indexes corresponding to paragraphs that best represent the diverse and critical topics in cleantech. Our selection criteria focused on the relevance of the content, its informativeness, and its potential to generate high-quality questions and answers.


***Process Overview:***

  - Domain Filtering:

      We separated the data into 5 primary domains: "azocleantech" ,"solarindustrymag", "cleantechnica", "pv-magazine" and "energyvoice". This ensures that we work with articles that are directly relevant to our project.

  - Manual Selection by Index:

      After filtering by domain, we manually inspected the articles and noted the indexes of the paragraphs with the highest quality content. For example, for the “azocleantech” domain, we selected 7 paragraphs, and for “solarindustrymag” we selected 8 paragraphs and we saved them in a csv file.

      These selections represent the most informative paragraphs that cover diverse topics in cleantech—ensuring that our subsequent QA generation process is based on rich and varied input.

  - Language Filter:

    To ensure consistency and adhere to the pattern demonstrated in the tutorial, we decided to work exclusively with English examples. This decision helps maintain focus on language that is well-supported by our NLP models. For more details on the tutorial’s approach, please refer to the [Cleantech RAG Tutorial](https://github.com/LuciferUchiha/Cleantech-RAG/blob/main/cleantech_rag.ipynb)

In [22]:
azocleantech_df = articles_df[articles_df['domain']=='azocleantech']

In [23]:
solarindustrymag_df = articles_df[articles_df['domain']=='solarindustrymag']

In [24]:
pv_magazine_df = articles_df[articles_df['domain']=='pv-magazine']

In [25]:
energyvoice_df = articles_df[articles_df['domain']=='energyvoice']

In [26]:
cleantechnica_df = articles_df[articles_df['domain']=='cleantechnica']

###Paragraphs for azocleantech domain

In [27]:
# 7 paragraphs of azocleantech_df
selected_indices = [17690, 17915, 18349, 17715, 19247, 15788, 2195]
paragraphs_azocleantech_df = azocleantech_df.loc[selected_indices]

In [28]:
# paragraphs_azocleantech_df.head()

In [29]:
# paragraphs_azocleantech_df.to_csv("paragraphs_azocleantech_df.csv")

###Paragraphs for solarindustrymag domain

In [30]:
# 8 paragraphs of solarindustrymag_df

selected_indices = [14,4307,1513,2216,13545,13382,2344,1578]
paragraphs_solarindustrymag_df = solarindustrymag_df.loc[selected_indices]

###Paragraphs for pv-magzine domain

In [31]:
pv_magazine_indices=pv_magazine_df.sample(n=10,random_state=42).index
paragraphs_pv_magazine_df = pv_magazine_df.loc[pv_magazine_indices]
# paragraphs_pv_magazine[['title','content']]


###Paragraphs for energyvoice domain

In [32]:
energyvoice_indices=energyvoice_df.sample(n=10,random_state=42).index
paragraphs_energyvoice_df = energyvoice_df.loc[energyvoice_indices]
# paragraphs_energyvoice[['title','content']]

###Paragraphs for cleantechnica domain

In [33]:
cleantechnica_df = articles_df[articles_df['domain']=='cleantechnica']

In [34]:
filtered_indices = cleantechnica_df.index[cleantechnica_df["Unnamed: 0"].isin(
    [95438, 95392, 95363, 95261, 95173, 95163, 95133, 95106,
     95074, 94927, 94925, 94662, 94668, 94645, 95411]
)].tolist()

In [35]:
paragraphs_cleantechnica_df = cleantechnica_df.loc[filtered_indices]

In [36]:
combined_df = pd.concat([paragraphs_azocleantech_df,
                         paragraphs_solarindustrymag_df,
                         paragraphs_pv_magazine_df,
                         paragraphs_energyvoice_df,
                         paragraphs_cleantechnica_df],
                        ignore_index=True)

In [37]:
combined_df.to_csv("paragraphs_media_dataset.csv")

## 3. QA Pair Generation

We use the Llama 3.2 Instruct model to generate QA pairs from our selected paragraphs. The idea is that each generated question and answer is directly tied to a specific cleantech-related content segment, providing the necessary context for later analysis.

For the media dataset, the QA pairs will contain the following keys:

  - question: The generated question.

  - relevant_text: The text (paragraph) used as context.

  - answer: The generated answer.

  - article_url: The URL of the source article.

For the patent dataset, the QA pairs include more detailed metadata:

  - question: The generated question.

  - relevant_text: The abstract used as context.

  - answer: The generated answer.

  - title: The title of the patent.

  - publication_date: The patent publication date.

  - inventor: The inventor(s) associated with the patent.


Below is the updated function. It accepts an additional argument dataset_type which specifies whether the DataFrame is from the media dataset ("media") or from the patent dataset ("patent"). Depending on the value, the function will append the appropriate metadata to each QA pair.

In [73]:
from openai import OpenAI
import pandas as pd
import re

def generate_qa_pairs(df, target_qa_pairs=43, dataset_type="media"):
    """
    Generates QA pairs from a given DataFrame using Llama 3.2 Instruct.

    Parameters:
      df (pd.DataFrame): DataFrame containing the reference texts.
         For the media dataset, it is expected to have a 'content' column and 'url'.
         For the patent dataset, it is expected to have an 'abstract' column, 'title',
         'publication_date', and 'inventor'.
      target_qa_pairs (int): The target number of QA pairs to generate.
      dataset_type (str): Specifies the dataset type. Use "media" for the media dataset,
         and "patent" for the patent dataset.

    Returns:
      pd.DataFrame: DataFrame containing the generated QA pairs and the associated metadata.
    """
    qa_pairs = []

    for _, row in df.iterrows():
        if dataset_type.lower() == "media":
            context_text = row['content']
        elif dataset_type.lower() == "patent":
            context_text = row['abstract']
        else:
            raise ValueError("dataset_type must be either 'media' or 'patent'.")

        paragraphs = context_text.split("\n\n")

        for paragraph in paragraphs:
            if not paragraph.strip():
                continue

            prompt = (
            "Using the following text, generate 15 unique and meaningful question–answer pairs.\n\n"
            "Instructions:\n"
            "1. Questions should be varied: include what, who, when, where, why, and how as appropriate.\n"
            "2. Focus on important details, facts, or concepts mentioned in the text.\n"
            "3. Provide clear answers that explain the key idea without being overly brief or too long.\n"
            "4. Format each pair as follows:\n"
            "Q: [Your question here]\n"
            "A: [Your answer here]\n\n"
            "Separate each Q&A pair with a blank line.\n\n"
            f"Text:\n{paragraph.strip()}"
        )


            try:
                completion = client.chat.completions.create(
                    model="meta-llama/llama-3.2-3b-instruct",
                    messages=[{"role": "user", "content": prompt}],
                )

                response = completion.choices[0].message.content.strip()

                # Extract Q&A pairs from the response
                qa_text = re.findall(r"Q:\s*(.*?)\s*A:\s*(.*?)(?=\nQ:|\Z)", response, re.DOTALL)

                for question, answer in qa_text:
                    print("question", question)
                    print("answer", answer)
                    if not any(existing["question"] == question for existing in qa_pairs):
                        if dataset_type.lower() == "media":
                            qa_pair = {
                                "question": question.strip(),
                                "relevant_text": paragraph.strip(),
                                "answer": answer.strip(),
                                "article_url": row.get('url', '')
                            }
                        else:
                            qa_pair = {
                                "question": question.strip(),
                                "relevant_text": paragraph.strip(),
                                "answer": answer.strip(),
                                "title": row.get('title', ''),
                                "publication_date": row.get('publication_date', ''),
                                "inventor": row.get('inventor', '')
                            }

                        qa_pairs.append(qa_pair)

                    if len(qa_pairs) >= target_qa_pairs:
                        break

            except Exception as e:
                print(f"Error: {e}")
                continue

        if len(qa_pairs) >= target_qa_pairs:
            break

    return pd.DataFrame(qa_pairs)


In [40]:
combined_df.head()

Unnamed: 0  \
0       23302   
1       23338   
2       23355   
3       23307   
4       23380   

                                                                           title  \
0                      Renewable Energy Partnerships: Green Hydrogen Integration   
1  Asahi Kasei Starts Operation of Multi-Module Hydrogen Pilot Plant in Kawasaki   
2                                Global Warming Threatens Hudson Bay Polar Bears   
3                              Efficient Recycling Key to Widespread EV Adoption   
4              Multi-Junction Solar Cells Offer a New Path to Sustainable Energy   

         date  author  \
0  2024-05-01     NaN   
1  2024-05-15     NaN   
2  2024-06-14     NaN   
3  2024-05-02     NaN   
4  2024-08-12     NaN   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            

In [74]:

# paragraphs_solarindustrymag_df = pd.read_csv('/content/drive/MyDrive/SwissText SharedTask/paragraphs_solarindustrymag_df.csv')
combined_qa_df = generate_qa_pairs(combined_df, target_qa_pairs=800, dataset_type="media")
# solarindustrymag_qa_df.to_csv('solarindustrymag_qa_pairs.csv', index=False)
combined_qa_df.to_csv("/content/drive/MyDrive/SwissText_SharedTask/combined_media_qa_pairs.csv", index= False)
combined_qa_df.head()

question What is green hydrogen, and how is it produced?
answer Green hydrogen is a sustainable energy carrier produced by water electrolysis using renewable energy sources, making it a carbon-neutral alternative to traditional hydrogen production.

question Who are some of the key players involved in the development and adoption of green hydrogen?
answer Key players include companies like Siemens Energy, ITM Power, and H2Pro, as well as academia and governments, such as the European Union, which are collaborating to advance the technology and its applications.

question What are the main challenges associated with integrating green hydrogen into the energy grid?
answer The main challenges include technical hurdles such as lower energy efficiency, safe storage and transportation, economic costs, and regulatory hurdles, as well as the need for significant infrastructure development.

question What is the role of the European Union's Hydrogen Strategy in promoting green hydrogen adoption

question  \
0                                                                      What is green hydrogen, and how is it produced?   
1                          Who are some of the key players involved in the development and adoption of green hydrogen?   
2                        What are the main challenges associated with integrating green hydrogen into the energy grid?   
3                     What is the role of the European Union's Hydrogen Strategy in promoting green hydrogen adoption?   
4  How does the Hydrogen Council facilitate partnerships and collaboration among companies, governments, and academia?   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 

In [ ]:
# azocleantech_qa_df  = generate_qa_pairs(paragraphs_solarindustrymag_df, target_qa_pairs=300, dataset_type="media")
# solarindustrymag_qa_df.to_csv('solarindustrymag_qa_pairs.csv', index=False)
solarindustrymag_qa_df  = pd.read_csv("/content/drive/MyDrive/SwissText SharedTask/azocleantech_qa_pairs.csv")
solarindustrymag_qa_df.head()

### Generate the QA Pairs for the Patent Dataset
For the patent dataset, we begin by filtering the articles based on keywords relevant to cleantech (using two keyword sets for azocleantech and solarindustrymag). Patents do not come pre-categorized into domains like media articles, so we must generate QA pairs first and then apply a separate categorization process.

To extract patents relevant to cleantech topics, we use a deterministic, rule-based approach.

- We start by defining two lists of keywords:
- Next, we apply these lists to the patent dataset by examining the abstract field of each patent. Using a lambda function with the apply method, we deterministically check if any keyword exists in an abstract. This yields two Boolean masks indicating whether a patent belongs to the azocleantech or solarindustrymag domain, respectively. We then filter the dataset based on these masks.
- We decided to use a deterministic, rule-based approach because we can directly see which keywords triggered the classification, allowing us to easily audit and refine our keyword lists if needed.
- However, we are aware that this method is rigid, so if an abstract uses synonyms or slightly different phrasing that isn’t in our keyword list, it might not be classified correctly. The approach does not consider context or the possibility of words having multiple meanings. An abstract might mention a keyword in a context that does not pertain to cleantech.

By using this deterministic approach, we ensure that our patent dataset is filtered in a consistent and transparent manner. While this method is efficient and straightforward, it does have limitations regarding flexibility and nuance.

In [ ]:
azocleantech_words = ['cleantech', 'clean energy', 'renewable', 'sustainable','technology', 'innovation', 'startup', 'investment','energy', 'power', 'grid', 'storage', 'battery']
solarindustrymag_words = ['solar', 'photovoltaic', 'PV', 'panel', 'module', 'installation', 'inverter', 'system', 'cells']

In [ ]:
patent_df = patent_df[patent_df['country_code']=="US"]

In [ ]:
mask_azocleantech = patent_df['abstract_localized'].apply(
    lambda text: any(keyword in text[0]['text'].lower() for keyword in azocleantech_words)
)
mask_solarindustrymag = patent_df['abstract_localized'].apply(
    lambda text: any(keyword in text[0]['text'].lower() for keyword in solarindustrymag_words)
)


df_azocleantech = patent_df[mask_azocleantech]
df_solarindustrymag = patent_df[mask_solarindustrymag]

In [ ]:
# azocleantech : 8569 	19340, 170 	361 , 10213 	23026, 952 	2044, 7021 	15812, 8000 	18020 , 1163 	2610
azocleantech_indexes = [19340,361,23026,2044,15812,18020,2610]

In [ ]:
azocleantech_df_selected_abstracts = df_azocleantech.loc[azocleantech_indexes]


In [ ]:
azocleantech_df_selected_abstracts

In [ ]:
azocleantech_df_selected_abstracts.to_csv('azocleantech_df_selected_abstracts.csv',index=False)

In [ ]:

abstracts_azocleantech_related = pd.read_csv('azocleantech_df_selected_abstracts.csv')
azocleantech_patents_qa_df = generate_qa_pairs(abstracts_azocleantech_related, target_qa_pairs=10, dataset_type="patent")
azocleantech_patents_qa_df.to_csv('azocleantech_patents_qa_df.csv', index=False)

In [ ]:
# solarindustrymag ->
solarindustrymag_indexes = [ 1, 29674 ,25783, 24035,22427,366,23331,19646]

In [ ]:
solarindustrymag_selected_abstracts = df_solarindustrymag.loc[solarindustrymag_indexes]

In [ ]:
solarindustrymag_selected_abstracts

In [ ]:
# solarindustrymag_selected_abstracts.to_csv('solarindustrymag_selected_abstracts.csv',index = False )

In [ ]:

# abstracts_solarindustrymag_related = pd.read_csv('solarindustrymag_selected_abstracts.csv')
# abstracts_paragraphs_azocleantech_related = pd.read_csv('azocleantech_df_selected_abstracts.csv')



In [ ]:
solarindustry_patents_qa_df = generate_qa_pairs(solarindustrymag_selected_abstracts, target_qa_pairs=10)


In [ ]:
solarindustry_patents_qa_df

In [ ]:
azocleantech_patents_qa_df = generate_qa_pairs(azocleantech_df_selected_abstracts, target_qa_pairs=10)



In [ ]:
azocleantech_patents_qa_df

In [ ]:
# solarindustry_patents_qa_df.to_csv('solarindustry_patents_qa_df.csv', index=False)
# azocleantech_patents_qa_df.to_csv('azocleantech_patents_qa_df.csv', index=False)

In [ ]:
# solarindustry_qa_df = pd.read_csv('/content/drive/MyDrive/SwissText SharedTask/solarindustrymag_qa_pairs.csv')
# azocleantech_qa_df = pd.read_csv('/content/drive/MyDrive/SwissText SharedTask/azocleantech_qa_pairs.csv')

In [ ]:
# solarindustry_patents_qa_df = pd.read_csv('/content/drive/MyDrive/SwissText SharedTask/solarindustry_patents_qa_df.csv')
# azocleantech_patents_qa_df = pd.read_csv('/content/drive/MyDrive/SwissText SharedTask/azocleantech_patents_qa_df.csv')

## 4. QA Pair Classification

After generating our QA pairs, our next step is to classify each question into a category. This categorization is essential to ensure that our QA pairs cover diverse aspects of cleantech, such as factual details, comparative market analysis, analytical explanations, government or corporate initiatives, and sustainability or technological innovations.

For each generated QA pair, we want to automatically determine the most suitable category based on the question's content. We experimented with two methods:

  -  A Rule-Based Approach:

        This method involves defining a dictionary of categories along with associated keywords. For each question, we count the occurrences of each keyword corresponding to every category. The category with the highest score is assigned to the question.
        It is easily interpretable since we can trace the category decision to keyword matches. However, it is limited flexibility if questions use synonyms or varied phrasing that is not in the keyword list.

  - BERT Sentence Approach:
    In this approach, we use a SentenceTransformer model to encode each generated question into an embedding vector. We then compare these embeddings to precomputed embeddings of category prototype texts (i.e., descriptions of what each category should capture) using cosine similarity. The category with the highest similarity score is assigned to the question.
    In this approach, Tthe quality of classification heavily depends on the choice of the prototype texts and the underlying model.
    We have evaluated both methods and found that the BERT sentence approach outperformed the rule-based method. For patent questions, we even experimented with replacing the  [all-MiniLM-L6-v2](https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2) model with the [AI-Growth-Lab/PatentSBERTa](https://github.com/AI-Growth-Lab/PatentSBERTa) model, which gave even better results in capturing nuanced technical details.



### The rule based approach

In [ ]:

# solarindustry_qa_df, azocleantech_qa_df, solarindustry_patents_qa_df, azocleantech_patents_qa_df

def classify_question_rule_based(question, categories):
    question_lower = question.lower()
    scores = {cat: 0 for cat in categories}
    for cat, keywords in categories.items():
        for kw in keywords:
            if kw in question_lower:
                scores[cat] += 1
    return max(scores, key=scores.get)

categories = {
    "Factual Questions": ["what", "when", "where", "who", "which", "fact"],
    "Comparative & Market Analysis Questions": ["compare", "contrast", "market", "trend", "analysis"],
    "Analytical & Explanatory Questions": ["why", "how", "explain", "impact"],
    "Government & Corporate Initiatives": ["government", "policy", "corporate", "initiative", "regulation"],
    "Sustainability & Technological Innovation Questions": ["sustainable", "innovation", "technology", "renewable", "eco-friendly"]
}

def stack_and_categorize(dfs, categories):
    df_combined = pd.concat(dfs, ignore_index=True)
    df_combined['category'] = df_combined['question'].apply(lambda q: classify_question_rule_based(q, categories))
    return df_combined




In [ ]:

dfs = [solarindustrymag_qa_df, solarindustrymag_qa_df, solarindustry_patents_qa_df, azocleantech_patents_qa_df]

df_categorized_rule_based = stack_and_categorize(dfs, categories)

print(df_categorized_rule_based.head())

### The Sentence transformer approach

#### The all-MiniLM-L6-v2 approach

In [ ]:
from sentence_transformers import SentenceTransformer, util
import torch

prototypes = {
    "Factual Questions": "Questions that ask for specific facts or details.",
    "Comparative & Market Analysis Questions": "Questions that compare options or analyze market trends.",
    "Analytical & Explanatory Questions": "Questions that require detailed analysis or explanation.",
    "Government & Corporate Initiatives": "Questions related to government policies or corporate initiatives.",
    "Sustainability & Technological Innovation Questions": "Questions focusing on sustainability and innovative technology."
}

model = SentenceTransformer('all-MiniLM-L6-v2')

prototype_texts = list(prototypes.values())
prototype_labels = list(prototypes.keys())
prototype_embeddings = model.encode(prototype_texts, convert_to_tensor=True)

def classify_question_embedding(question: str) -> str:
    """
    Classify a question into one of the five categories using cosine similarity
    between its embedding and the precomputed prototype embeddings.
    """
    question_embedding = model.encode(question, convert_to_tensor=True)
    cosine_scores = util.cos_sim(question_embedding, prototype_embeddings)
    best_idx = torch.argmax(cosine_scores).item()
    return prototype_labels[best_idx]

def stack_and_categorize(dfs: list) -> pd.DataFrame:
    """
    Stack multiple DataFrames and add a new 'category' column by classifying each question.
    """
    df_combined = pd.concat(dfs, ignore_index=True)
    df_combined['category'] = df_combined['question'].apply(lambda q: classify_question_embedding(q))
    return df_combined



dfs = [
    solarindustrymag_qa_df,
    solarindustrymag_qa_df,
    solarindustry_patents_qa_df,
    azocleantech_patents_qa_df
]

df_categorized_sentence_bert = stack_and_categorize(dfs)


In [ ]:
df_categorized_sentence_bert

#### The AI-Growth-Lab/PatentSBERTa approach

In [ ]:

model = SentenceTransformer('AI-Growth-Lab/PatentSBERTa')

prototype_texts = list(prototypes.values())
prototype_labels = list(prototypes.keys())
prototype_embeddings = model.encode(prototype_texts, convert_to_tensor=True)


df_categorized_patent_sentence_bert = stack_and_categorize(dfs)



### Visualizing and Comparing Classification Approaches

After classifying our QA pairs, we plotted the number of questions in each category to evaluate how well each approach captured the variety of question types.

  1. Rule-Based Approach
  The rule-based approach appears biased toward Factual Questions because it heavily relies on keyword matching. When questions do not contain the keywords associated with the other categories, they are often labeled as factual by default.

  2. BERT Sentence Approach
  The BERT based semantic classification captures more nuance and distributes questions more evenly across multiple categories. It identifies many questions about sustainability and technological innovation highlighting how semantic embeddings can detect relevant context even if explicit keywords are missing.

  3. Patent-Specific BERT Model (PatentSBERTa)
  Because PatentSBERTa is fine-tuned for patent text, it performs even better at placing technical questions into the correct categories. This is especially important for recognizing subtle phrases and context in patent abstracts.


We decided to go with the Bert approach for classifying all QA pairs, ensuring more nuanced and reliable coverage of the different cleantech question types.

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
categories_rule_based = df_categorized_rule_based['category'].value_counts()


In [ ]:
barplot = sns.barplot(
    x=categories_rule_based.values,
    y=categories_rule_based.index,
    hue=categories_rule_based.index
)

barplot.set_title('Question Count by Categories')
barplot.set_xlabel('Question Count')
barplot.set_ylabel('Category')

plt.show()

In [ ]:
categories_sentence_bert = df_categorized_sentence_bert['category'].value_counts()

barplot = sns.barplot(
    x=categories_sentence_bert.values,
    y=categories_sentence_bert.index,
    hue=categories_sentence_bert.index
)

barplot.set_title('Question Count by Categories')
barplot.set_xlabel('Question Count')
barplot.set_ylabel('Category')

plt.show()

In [ ]:
categories_patent_bert = df_categorized_patent_sentence_bert['category'].value_counts()

barplot = sns.barplot(
    x=categories_patent_bert.values,
    y=categories_patent_bert.index,
    hue=categories_patent_bert.index
)

barplot.set_title('Question Count by Categories')
barplot.set_xlabel('Question Count')
barplot.set_ylabel('Category')

plt.show()

## The QAs pairs are divided in these questions categories:

- Factual Questions

- Comparative & Market Analysis Questions

- Analytical & Explanatory Questions

- Government & Corporate Initiatives

- Sustainability & Technological Innovation Questions



### Factual Questions

Factual Questions: Ask for concrete details or definitions (e.g., “What is…”).

In [ ]:
factual_questions = df_categorized_sentence_bert[df_categorized_sentence_bert['category'] == 'Factual Questions']['question'].tolist()
factual_df = df_categorized_sentence_bert[df_categorized_sentence_bert['category'] == 'Factual Questions']


In [ ]:
show_questions("Factual Questions", factual_questions)


In [ ]:
plot_character_distribution(factual_df)
plot_word_distribution(factual_df)
plot_gpt4_token_distribution(factual_df)
plot_non_stopword_token_distribution(factual_df)

### Comparative & Market Analysis Questions

Involve comparing cleantech technologies or approaches (e.g., “What are the differences between…”).

In [ ]:
Comparative_Market_Analysis_Questions = df_categorized_sentence_bert[df_categorized_sentence_bert['category'] == 'Comparative & Market Analysis Questions']['question'].tolist()
Comparative_Market_Analysis_df = df_categorized_sentence_bert[df_categorized_sentence_bert['category'] == 'Comparative & Market Analysis Questions']


In [ ]:
show_questions("Comparative & Market Analysis Questions", Comparative_Market_Analysis_Questions)


In [ ]:
plot_character_distribution(Comparative_Market_Analysis_df)
plot_word_distribution(Comparative_Market_Analysis_df)
plot_gpt4_token_distribution(Comparative_Market_Analysis_df)
plot_non_stopword_token_distribution(Comparative_Market_Analysis_df)

### Analytical & Explanatory Questions

equire reasoning or explanation (e.g., “How does…”).

In [ ]:
Analytical_Explanatory_Questions = df_categorized_sentence_bert[df_categorized_sentence_bert['category'] == 'Analytical & Explanatory Questions']['question'].tolist()
Analytical_Explanatory_Questions_df = df_categorized_sentence_bert[df_categorized_sentence_bert['category'] == 'Analytical & Explanatory Questions']

In [ ]:
show_questions("Analytical & Explanatory Questions", Analytical_Explanatory_Questions)


In [ ]:
plot_character_distribution(Analytical_Explanatory_Questions_df)
plot_word_distribution(Analytical_Explanatory_Questions_df)
plot_gpt4_token_distribution(Analytical_Explanatory_Questions_df)
plot_non_stopword_token_distribution(Analytical_Explanatory_Questions_df)



### Government & Corporate Initiatives

In [ ]:
Government_Corporate_Initiatives = df_categorized_sentence_bert[df_categorized_sentence_bert['category'] == 'Government & Corporate Initiatives']['question'].tolist()
Government_Corporate_Initiatives_df = df_categorized_sentence_bert[df_categorized_sentence_bert['category'] == 'Government & Corporate Initiatives']

In [ ]:
show_questions("Government & Corporate Initiatives", Government_Corporate_Initiatives)


In [ ]:
plot_character_distribution(Government_Corporate_Initiatives_df)
plot_word_distribution(Government_Corporate_Initiatives_df)
plot_gpt4_token_distribution(Government_Corporate_Initiatives_df)
plot_non_stopword_token_distribution(Government_Corporate_Initiatives_df)

### Sustainability & Technological Innovation Questions

In [ ]:
Sustainability_Technological_Innovation_Questions = df_categorized_sentence_bert[df_categorized_sentence_bert['category'] == 'Sustainability & Technological Innovation Questions']['question'].tolist()
Sustainability_Technological_Innovation_Questions_df = df_categorized_sentence_bert[df_categorized_sentence_bert['category'] == 'Sustainability & Technological Innovation Questions']

In [ ]:
show_questions("Sustainability & Technological Innovation Questions", Sustainability_Technological_Innovation_Questions)


In [ ]:
plot_character_distribution(Sustainability_Technological_Innovation_Questions_df)
plot_word_distribution(Sustainability_Technological_Innovation_Questions_df)
plot_gpt4_token_distribution(Sustainability_Technological_Innovation_Questions_df)
plot_non_stopword_token_distribution(Sustainability_Technological_Innovation_Questions_df)

## 5. Setup critique agents

The questions generated by our LLM may exhibit various flaws. For instance, some questions might lack sufficient context to be answered accurately, and others may depend too much on the original document context, making them unclear when viewed in isolation.

To address this, we build critique agents—automated evaluators that rate each QA pair on three key criteria:

  - Groundedness:
  Can the question be answered unambiguously given the provided context?

  - Relevance:
  Is the question useful to machine learning developers building NLP applications?
  (For example, a question like “What is the date when transformers 4.29.1 was released?” is not very relevant.)

  - Standalone Clarity:
  Does the question make sense on its own without requiring additional context?
  (A question that refers implicitly to a specific setting without clarification would score poorly here.)

These criteria are inspired by the evaluation methods described in this [paper](https://arxiv.org/abs/2312.10003). To obtain a reliable judgment, we prompt the LLM with instructions that ask for not only a numerical score (on a scale from 1 to 5) but also a detailed rationale for that score. Providing the rationale helps verify the scoring process and gives the model additional tokens to “think” about the answer before summarizing it.

Below is the code that sets up the critique agents and applies them to our generated QA pairs.

In [ ]:
media_qa = pd.read_csv("/content/drive/MyDrive/SwissText SharedTask/merged_media_qa_pairs.csv")
patent_qa = pd.read_csv("/content/drive/MyDrive/SwissText SharedTask/patent_dataset_qa_pairs.csv")

### Merge the 2 datasets and categorize the questions

In [ ]:

dfs = [
    media_qa,
    patent_qa
]

df_categorized_sentence_bert = stack_and_categorize(dfs)


In [ ]:
df_categorized_sentence_bert.to_csv('QA_pairs_categorized.csv')

In [ ]:
# convert the df to langchain document
outputs = [LangchainDocument(page_content=row['relevant_text'], metadata =
 {'question':row['question'],'answer':row['answer'], 'category':row['category'], 'title':row['title'],'url':row['article_url']})
 for idx, row in df_categorized_sentence_bert.iterrows()]


In [ ]:
outputs = []
for idx, row in df_categorized_sentence_bert.iterrows():
    outputs.append({
        "context": row['relevant_text'],
        "question": row['question'],
        "answer": row['answer'],
        "source_doc": row['article_url'],
        "category": row["category"]
    })


In [ ]:
len(outputs)

In [ ]:
def remove_dublicates(outputs):
  seen_content = {}
  unique_outputs = []
  for doc in outputs:
      content_question_pair = (doc['context'], doc['question'])
      # content_question_pair = (doc.page_content, doc.metadata['question'])
      if content_question_pair not in seen_content:
                seen_content[content_question_pair] = True
                unique_outputs.append(doc)

  return unique_outputs

outputs = remove_dublicates(outputs)

In [ ]:
len(outputs)

### Critique promts for evaluation

In [ ]:
question_groundedness_critique_prompt = """
You will be given a context and a question.
Your task is to provide a 'total rating' scoring how well one can answer the given question unambiguously with the given context.
Give your answer on a scale of 1 to 5, where 1 means that the question is not answerable at all given the context, and 5 means that the question is clearly and unambiguously answerable with the context.

Provide your answer as follows:

Answer:::
Evaluation: (your rationale for the rating, as a text)
Total rating: (your rating, as a number between 1 and 5)

You MUST provide values for 'Evaluation:' and 'Total rating:' in your answer.

Now here are the question and context.

Question: {question}\n
Context: {context}\n
Answer::: """

question_relevance_critique_prompt = """
You will be given a question.
Your task is to provide a 'total rating' representing how useful this question can be to machine learning developers building NLP applications with the Hugging Face ecosystem.
Give your answer on a scale of 1 to 5, where 1 means that the question is not useful at all, and 5 means that the question is extremely useful.

Provide your answer as follows:

Answer:::
Evaluation: (your rationale for the rating, as a text)
Total rating: (your rating, as a number between 1 and 5)

You MUST provide values for 'Evaluation:' and 'Total rating:' in your answer.

Now here is the question.

Question: {question}\n
Answer::: """

question_standalone_critique_prompt = """
You will be given a question.
Your task is to provide a 'total rating' representing how context-independent this question is.
Give your answer on a scale of 1 to 5, where 1 means that the question depends on additional information to be understood, and 5 means that the question makes sense by itself.
For instance, if the question refers to a particular setting, like 'in the context' or 'in the document', the rating must be 1.
The questions can contain obscure technical nouns or acronyms like Gradio, Hub, Hugging Face or Space and still be a 5: it must simply be clear to an operator with access to documentation what the question is about.

For instance, "What is the name of the checkpoint from which the ViT model is imported?" should receive a 1, since there is an implicit mention of a context, thus the question is not independent from the context.

Provide your answer as follows:

Answer:::
Evaluation: (your rationale for the rating, as a text)
Total rating: (your rating, as a number between 1 and 5)

You MUST provide values for 'Evaluation:' and 'Total rating:' in your answer.

Now here is the question.

Question: {question}\n
Answer::: """

In [ ]:
import re

# Pattern to capture evaluation text and score:
pattern = re.compile(
    r"Evaluation:\s*(.*?)\s*Total rating:\s*(\d+)",
    re.DOTALL,
)


In [ ]:
from openai import OpenAI

client = OpenAI(
  base_url="https://openrouter.ai/api/v1",
  api_key="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",
)

def call_llm(client: OpenAI, prompt: str):
    completion = client.chat.completions.create(
    extra_body={},
    model="meta-llama/llama-3.2-3b-instruct",
      # model="meta-llama/llama-3.2-3b-instruct:free",
    messages=[
      {
        "role": "user",
        "content": prompt
      }
    ]
  )
    if not completion.choices:
      print(completion)
      return None
    return completion.choices[0].message.content

# result = call_llm(client, "This is a  context")
# print(result)


In [ ]:
def create_eval(outputs):
    tests = []
    for i in range(0, len(outputs), 100):
      for output in tqdm(outputs[i:i+100]):
        eval_result = {
            "context": output["context"],
            "question": output["question"],
            "answer": output["answer"],
            "source_doc": output["source_doc"],
            "category": output["category"],
        }

        evaluations = {
            "groundedness": call_llm(
                client,
                question_groundedness_critique_prompt.format(
                    context=output["context"], question=output["question"]
                ),
            ),
            "relevance": call_llm(
                client,
                question_relevance_critique_prompt.format(question=output["question"]),
            ),
            "standalone": call_llm(
                client,
                question_standalone_critique_prompt.format(question=output["question"]),
            ),
        }

        for criterion, evaluation in evaluations.items():
            match = pattern.search(evaluation)
            if match:
                eval_text = match.group(1).strip()
                score = int(match.group(2).strip())
                eval_result[f"{criterion}_score"] = score
                eval_result[f"{criterion}_eval"] = eval_text
            else:
                print(f"{criterion} evaluation did not match:")
                print(evaluation)
                continue

        tests.append(eval_result)

    return pd.DataFrame.from_dict(tests)


In [ ]:
results = []

In [ ]:
print("Generating critique for each QA couple...")
for i in range(0, len(outputs), 100):
  for output in tqdm(outputs[i:i+100]):
    eval_result = {
        "context": output["context"],
        "question": output["question"],
        "answer": output["answer"],
        "source_doc": output["source_doc"],
        "category": output["category"],
    }

    evaluations = {
        "groundedness": call_llm(
            client,
            question_groundedness_critique_prompt.format(
                context=output["context"], question=output["question"]
            ),
        ),
        "relevance": call_llm(
            client,
            question_relevance_critique_prompt.format(question=output["question"]),
        ),
        "standalone": call_llm(
            client,
            question_standalone_critique_prompt.format(question=output["question"]),
        ),
    }

    for criterion, evaluation in evaluations.items():
        match = pattern.search(evaluation)
        if match:
            eval_text = match.group(1).strip()
            score = int(match.group(2).strip())
            eval_result[f"{criterion}_score"] = score
            eval_result[f"{criterion}_eval"] = eval_text
        else:
            print(f"{criterion} evaluation did not match:")
            print(evaluation)
            continue

    results.append(eval_result)

generated_questions = pd.DataFrame.from_dict(results)
generated_questions.to_csv('QA_cleaned')


In [ ]:
generated_questions.to_csv('QA_cleaned.csv')


In [ ]:
len(generated_questions)

## 6. Filtering & Quality Control

After running our QA pairs through the LLM-based critique agents, we now have scores for each question on three key criteria: groundedness, relevance, and standalone clarity. In this step, we filter out the QA pairs that score below our threshold in any of these criteria. The following code displays the evaluation metrics before filtering and then retains only those questions that score at least 3 on each criterion.

In our evaluation, we began with a total of 694 QA pairs. After applying our filtering criteria—removing any pair that scored below 3 in groundedness, relevance, or standalone clarity—only 86 QA pairs remained. This significant reduction indicates that many generated QA pairs did not meet our quality standards. Such a high attrition rate may be due to the limitations of the generation process, where some questions might be ambiguous, not contextually grounded, or overly dependent on the source text. It could also reflect the sensitivity of our evaluation prompts and the complexity of the cleantech domain, particularly in technical patent contexts.

Although retaining only 86 out of 694 QA pairs might seem drastic, this rigorous filtering ensures that our final dataset contains only high-quality, reliable questions. By using only the top-performing QA pairs, we significantly reduce noise in our dataset, thereby establishing a robust foundation for subsequent stages of our RAG system. Ultimately, while the filtering process discards a large portion of the initially generated pairs, it enhances the overall quality, making our downstream tasks more efficient and effective.

In [ ]:
# generated_questions = pd.read_csv("/content/drive/MyDrive/SwissText SharedTask/QA_cleaned.csv")

In [ ]:
import pandas as pd

pd.set_option("display.max_colwidth", None)

def eval_qa_pairs(generated_questions):

  print("Evaluation dataset before filtering:")
  display(
      generated_questions[
          [
              "question",
              "answer",
              "groundedness_score",
              'groundedness_eval',
              "relevance_score",
              "relevance_eval",
              "standalone_score",
              "standalone_eval",
          ]
      ]
  )
  generated_questions_cleaned = generated_questions.loc[
      (generated_questions["groundedness_score"] >= 3)
      & (generated_questions["relevance_score"] >= 3)
      & (generated_questions["standalone_score"] >= 3)
  ]
  print("============================================")
  print("Final evaluation dataset:")
  display(
      generated_questions_cleaned[
          [
              "question",
              "answer",
              "groundedness_score",
              "relevance_score",
              "standalone_score",
          ]
      ]
  )

  return generated_questions_cleaned



In [ ]:
len(generated_questions)

In [ ]:
generated_questions_cleaned = eval_qa_pairs(generated_questions)

In [ ]:
len(generated_questions_cleaned)

In [ ]:
# generated_questions_cleaned.to_csv("/content/drive/MyDrive/SwissText SharedTask/generated_questions_cleaned.csv")

In [ ]:
# generated_questions_cleaned  = pd.read_csv("/content/drive/MyDrive/SwissText SharedTask/generated_questions_cleaned.csv")

From the set of five questions below, we observe several key characteristics that demonstrate the quality and diversity of the generated QA pairs. Overall, these examples illustrate that our process successfully produced questions that are contextually grounded, relevant, and thought-provoking:

  - The questions address both technical and real-world concerns—for example, one question explores the potential for recycling electric car batteries and its broader environmental and economic benefits, while another discusses the impact of global warming on wildlife. This shows that our QA pairs capture a range of topics within the cleantech domain.

  - They integrate multiple aspects of the domain. Questions connect technological aspects (like recycling challenges and grid stability with V2X technology) with behavioral or market trends (such as the influence of owning an electric vehicle on solar panel installations). This blend of technical and contextual elements makes the QA pairs more comprehensive.

  - The questions are formulated to invite detailed and informative answers. By asking about challenges and potential solutions in one case or the broader implications of a technology in another, these questions encourage responses that are both analytical and forward-looking.

  - In general, the generated questions are clear and self-contained, meaning they can stand alone without needing extensive external context. This quality is crucial for ensuring that the QA pairs are useful in downstream tasks, like feeding into a retrieval-augmented generation system.

In summary, the diversity and clarity of these questions from technical specifics to broader market and environmental implications demonstrate that our QA pair generation method is effective at producing high-quality, varied, and contextually rich questions across different aspects of cleantech innovation.

  

These questions collectively show that our approach results in a robust QA dataset that captures various dimensions of the cleantech field.


In [ ]:

generated_questions_cleaned['question'].head()

## 7. Evaluation Distributions

In [ ]:
def plot_evaluation_distribution(df, score_field, title):
    plt.figure(figsize=(8, 4))
    sns.histplot(df[score_field], bins=5, kde=False)
    plt.title(title)
    plt.xlabel("Score")
    plt.ylabel("Frequency")
    plt.xticks(range(1, 7))
    plt.show()

plot_evaluation_distribution(generated_questions_cleaned, "groundedness_score", "Distribution of Groundedness Scores")
plot_evaluation_distribution(generated_questions_cleaned, "relevance_score", "Distribution of Relevance Scores")
plot_evaluation_distribution(generated_questions_cleaned, "standalone_score", "Distribution of Standalone Scores")


### Evaluation of the Human Evaluation Dataset Analysis

In addition to our generated QA pairs, we evalutate the human evaluation dataset(cleantech_rag_evaluation_data_2024-09-20.csv). This dataset is a small, high-quality evaluation set designed to benchmark the retrieval step in the RAG system. The human evaluators have already curated these 28 examples for quality.

We pass this dataset through our classification and evaluation pipeline to see how well our automated QA evaluation compares. After running the examples through the pipeline (which includes our LLM-based critique agents and filtering logic), we found that only 2 out of the 28 examples were classified as good.

This outcome highlights a few key insights:

  - High Standards for Quality:
  Our pipeline uses strict evaluation metrics for groundedness, relevance, and standalone clarity. The fact that only 2 examples met our threshold indicates that these metrics are effective at distinguishing high quality QA pairs from those that are ambiguous, irrelevant, or overly context dependent.

  - Differences in Human vs. Automated Judgment:
  Although the human evaluation dataset is curated for quality, our automated critique agents may apply slightly different standards. This gap can arise due to the inherent differences between human judgment and the deterministic nature of our LLM-based scoring.

  - Potential Areas for Improvement:
  The low pass rate (2/28) suggests that even the high-quality human-curated examples can sometimes fall short of our strict automatic evaluation criteria. This may be due to subtle issues in the questions or context that our models pick up on. It also indicates that our QA generation and evaluation techniques could be further fine-tuned to align better with human judgments.

  - Implications for the Retrieval Step:
  Since this evaluation dataset is specifically for assessing the Retrieval component of our RAG system, maintaining a high-quality benchmark is crucial. Even though our automated pipeline filters aggressively, ensuring that the retrieval component consistently returns the top-quality examples is a key factor in the overall system performance.

In summary, processing the human evaluation dataset demonstrates the robustness of our filtering and quality control process, while also revealing areas where our automated system might differ from human expectations. This insight is valuable for tuning both our QA generation methods and our evaluation criteria to create an optimal RAG system.


In [ ]:
human_eval

In [ ]:
# human_eval = pd.read_csv("/content/drive/MyDrive/SwissText SharedTask/human_eval_csv.csv")

In [ ]:

dfs = [human_eval]

human_eval_categorized_sentence_bert = stack_and_categorize(dfs)

In [ ]:
human_eval_categorized_sentence_bert

In [ ]:
outputs = []
for idx, row in human_eval_categorized_sentence_bert.iterrows():
    outputs.append({
        "context": row['relevant_text'],
        "question": row['question'],
        "answer": row['answer'],
        "source_doc": row['article_url'],
        "category": row["category"]
    })


In [ ]:
results = create_eval(outputs)

In [ ]:
evaluated_results = eval_qa_pairs(results)

In [ ]:
len(evaluated_results)

## 8. Conclusion & Next Steps

In this notebook, we explored the process of creating and analyzing QA pairs from 50–100 carefully selected paragraphs from the Cleantech Media and Google Patent datasets. By generating, classifying, and rigorously evaluating these QA pairs using LLMs, we built a high-quality dataset that serves as a robust foundation for our RAG pipeline.

Next Steps:

  - Integrate the filtered QA dataset into the implemeted RAG system.

  - Experiment with different LLM prompts and filtering thresholds to refine your QA pair quality.

  - Use the insights gained from the evaluation visualizations to further tune your retrieval and generation processes.

In [ ]:
articles_df.columns

In [ ]:
patent_df.columns

In [ ]:
len(articles_df)

In [ ]:
from openai import OpenAI

client = OpenAI(
  base_url="https://openrouter.ai/api/v1",
  api_key="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",
)

def generate_qa_pairs(df, target_qa_pairs=43):

    client = OpenAI(
    base_url="https://openrouter.ai/api/v1",
    api_key="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",
    )

    qa_pairs = []
    # for _ in range(target_qa_pairs):
    for _, row in df.iterrows():
        abstract = row['content']
        prompt = f"""
                Based on the following reference text, generate a relevant question and its corresponding answer.
                The generated question must meet all the following quality criteria:

                1. Groundedness: The question must be clearly answerable using only the information provided in the reference text.
                2. Relevance: The question must be useful and interesting for machine learning practitioners, especially for those building NLP applications. Avoid overly trivial or time-sensitive questions.
                3. Stand-alone: The question must be self-contained and understandable without additional context or reference to the document it came from. It should make sense to anyone with general domain knowledge.

                Please ensure the question is phrased in a way that clearly adheres to these criteria. Use the following guidelines as a reference:
                - If the question refers only to details provided in the text, it is considered grounded.
                - If the question can be useful in practical scenarios (such as in the Hugging Face ecosystem), it is considered relevant.
                - If the question is understandable without needing extra context (e.g., it does not include phrases like "in the document" or "as mentioned above"), it is considered stand-alone.

                Now, here is the reference text:

                {abstract}

                Provide your output in the following format:

                Question: <generated question>
                Answer: <generated answer>
                """

        completion = client.chat.completions.create(
            extra_body={},
            model="meta-llama/llama-3.2-3b-instruct:free",
            messages=[
                {
                    "role": "user",
                    "content": prompt
                }
            ]
        )


      # if completion.choices:
        try:
          qa_text = completion.choices[0].message.content.split("**")
          if len(qa_text) >= 2 :
            question = qa_text[0]
            answer = qa_text[1]
            qa_pairs.append({
                "question": question,
                "relevant_text": abstract,
                "answer": answer,
                "url": row.get('url','None'),
            })
          else: continue
        except Exception as e:
          print(completion)

    return pd.DataFrame(qa_pairs)


In [ ]:
random_articles_df_sample = articles_df.sample(n=200, random_state=42)
random_patent_df_sample = patent_df.sample(n=200, random_state=42)

In [ ]:
articles_qa_sample = generate_qa_pairs(random_articles_df_sample, target_qa_pairs=43)

In [ ]:
len(articles_qa_sample)

In [ ]:
random_articles_df_sample.iloc[43]['content']

In [ ]:
abstract = random_articles_df_sample.iloc[43]['content']
qa_pairs = []

client = OpenAI(
  base_url="https://openrouter.ai/api/v1",
  api_key="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",
)

prompt = f"""
        Based on the following reference text, generate a relevant question and its corresponding answer.
        The generated question must meet all the following quality criteria:

        1. Groundedness: The question must be clearly answerable using only the information provided in the reference text.
        2. Relevance: The question must be useful and interesting for machine learning practitioners, especially for those building NLP applications. Avoid overly trivial or time-sensitive questions.
        3. Stand-alone: The question must be self-contained and understandable without additional context or reference to the document it came from. It should make sense to anyone with general domain knowledge.

        Please ensure the question is phrased in a way that clearly adheres to these criteria. Use the following guidelines as a reference:
        - If the question refers only to details provided in the text, it is considered grounded.
        - If the question can be useful in practical scenarios (such as in the Hugging Face ecosystem), it is considered relevant.
        - If the question is understandable without needing extra context (e.g., it does not include phrases like "in the document" or "as mentioned above"), it is considered stand-alone.

        Now, here is the reference text:

        {abstract}

        Provide your output in the following format:

        Question: <generated question>
        Answer: <generated answer>
        """

completion = client.chat.completions.create(
    extra_body={},
    model="meta-llama/llama-3.2-3b-instruct:free",
    messages=[
        {
            "role": "user",
            "content": prompt
        }
    ]
)


# if completion.choices:
print(completion)
qa_text = completion.choices[0].message.content.split("\n")
print()
print(qa_text)
print(len(qa_text))


if len(qa_text) >= 2 :
  question = qa_text[0]
  answer = qa_text[1]
  qa_pairs.append({
      "question": question,
      "relevant_text": abstract,
      "answer": answer,
      "url": row.get('url','None'),
  })
else:
  print(qa_text)


In [ ]:
qa_pairs[0]['answer']

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


In [ ]:
!jupyter nbconvert --to html --template pj /content/drive/MyDrive/Colab Notebooks/QA_Pairs.ipynb
